In [5]:
from pyspark.sql        import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder\
      .getOrCreate()

spark.sparkContext._jsc\
     .hadoopConfiguration().set("mapreduce.outputcommitter.factory.scheme.s3a", "org.apache.hadoop.fs.s3a.commit.S3ACommitterFactory")
spark.sparkContext._jsc\
     .hadoopConfiguration().set("fs.s3a.committer.name", "directory")
spark.sparkContext._jsc\
      .hadoopConfiguration().set("fs.s3a.committer.staging.tmp.path", "/tmp/spark_staging")
spark.sparkContext._jsc\
      .hadoopConfiguration().set("fs.s3a.buffer.dir", "/tmp/spark_local_buf")
spark.sparkContext._jsc\
      .hadoopConfiguration().set("fs.s3a.committer.staging.conflict-mode", "fail")
spark.sparkContext._jsc\
      .hadoopConfiguration().set("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
spark.sparkContext._jsc\
      .hadoopConfiguration().set("fs.s3a.access.key", "minio_access_key")
spark.sparkContext._jsc\
      .hadoopConfiguration().set("fs.s3a.secret.key", "minio_secret_key")
spark.sparkContext._jsc\
      .hadoopConfiguration().set("fs.s3a.endpoint", "http://minio:9000")
spark.sparkContext._jsc\
      .hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")
spark.sparkContext._jsc\
      .hadoopConfiguration().set("fs.s3a.path.style.access", "true")


In [ ]:
#local para salvar csv
local_csv = "s3a://sicredi/etl_csv"

In [2]:
#lendo dados de origem

df_movimento = spark.read.parquet("s3a://sicredi/movimento")
df_cartao    = spark.read.parquet("s3a://sicredi/cartao")
df_conta     = spark.read.parquet("s3a://sicredi/conta")
df_associado = spark.read.parquet("s3a://sicredi/associado")

In [9]:
df_movimento_flat = df_movimento.drop("id").withColumnRenamed("vlr_transacao","vlr_transacao_movimento")\
                                           .withColumnRenamed("des_transacao","desc_transacao_movimento")\
                                           .join(df_cartao.selectExpr("id as id_cartao","nom_impresso as nome_impresso_cartao","num_cartao as numero_cartao","id_conta","id_associado"),"id_cartao","left")\
                                           .join(df_conta.selectExpr("id as id_conta","tipo as tipo_conta","data_criacao as data_criacao_conta"),"id_conta","left")\
                                           .join(df_associado.selectExpr("id as id_associado","nome as nome_associado","sobrenome as sobrenome_associado","idade as idade_associado"),"id_associado","left").drop("id_conta","id_cartao","id_associado")

In [10]:
df_movimento_flat.show()

+--------------+------------------------+-----------------------+--------------------+----------------+----------+------------------+--------------+-------------------+---------------+
|data_movimento|desc_transacao_movimento|vlr_transacao_movimento|nome_impresso_cartao|   numero_cartao|tipo_conta|data_criacao_conta|nome_associado|sobrenome_associado|idade_associado|
+--------------+------------------------+-----------------------+--------------------+----------------+----------+------------------+--------------+-------------------+---------------+
|    2022-09-12|                 mercado|                  700.9|         Renan Matos|9912930130918239|  corrente|        2022-05-12|         Renan|              Matos|             25|
|    2022-09-12|                farmacia|                   12.9|         Renan Matos|9912930130918239|  corrente|        2022-05-12|         Renan|              Matos|             25|
|    2022-09-12|                farmacia|                  128.9|         R

In [ ]:
df_movimento_flat.write.parquet("s3a://sicredi/movimento_flat")
df_movimento_flat.write.csv(local_csv)